# Evolutionary Algorithm for Debiasing Glove

## Fitness criteria is debiasing only

## Import Statements

In [1]:
from utils import *
from cluster import *
import codecs
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Load Glove Embedding

In [2]:
glove_wv, glove_w2i, glove_vocab = load_embedding('./data/glove.txt')
glove_wv = decentralize(glove_wv)
glove_wv = normalize(glove_wv)

## Set up clustering

In [3]:
# limit vocab by excluding words that 'should' have gender bias
gender_specific = []

with open('./data/male_words.txt') as f:
    for l in f:
        gender_specific.append(l.strip())
with open('./data/female_words.txt') as f:
    for l in f:
        gender_specific.append(l.strip())

with codecs.open('./data/gender_specific_full.json') as f:
    gender_specific.extend(json.load(f))
    
glove_vocab_limit, glove_wv_limit, glove_w2i_limit = limit_vocab(glove_wv, glove_w2i, glove_vocab, exclude=gender_specific)

# get most biased words
he_vector = glove_wv[glove_w2i['he'], :]
she_vector = glove_wv[glove_w2i['she'], :]
biased_words = compute_word_bias(glove_wv_limit, glove_w2i_limit, glove_vocab_limit, he_vector, she_vector)

## Set up categorization

In [4]:
from web.evaluate import evaluate_categorization
from web.datasets.categorization import fetch_BLESS

In [5]:
bless_data = fetch_BLESS()

## Evolutionary Algorithm functions

In [25]:
def mutation(parent):
    '''Mutate one point in the parent's genome ie. small mutation
    Input: parent array
    Output: child array
    '''
    
    child = np.copy(parent)
    
    # select one point on the embedding to mutate
    i = np.random.randint(0, len(parent))
    child[i] += np.random.normal(0, 0.01)
    
    return child


def crossover(parent1, parent2):
    '''Create one new child from two parents
    Input: two parent arrays
    Output: one child array
    '''
    
    child = np.zeros(len(parent1))
    
    # select random breakpoint on genome and combine parents
    i = np.random.randint(1, len(parent1)-1)
    child[:i] = parent1[:i]
    child[i:] = parent2[i:]
    
    return child


def fitness_based_selection(fitness_list):
    '''Given the list of fitness scores for the population, randomly select
    and index as if spinning a roulette wheel with space on the wheel proportional
    to the individual fitness scores
    Input: list of fitness scores
    Output: index of selected individual
    '''
    
    index = None
    
    # randomly place a pointer on roulette wheel
    s = sum(fitness_list)
    pointer = np.random.random() * s
    
    # keep adding fitness until it surpasses pointer location
    p = 0
    for i in range(len(fitness_list)):
        p += fitness_list[i]
        if p > pointer:
            index = i
            break
            
    return index


def evaluate_fitness(wv, w2i, vocab):
    '''Given an embedding, return the fitness where fitness is a combination
    of amount of debiasing and retained utility. We use clustering precision
    of most biased words as proxy for bias, so 1-precision is added to fitness.
    We use concept categorization as a proxy for utility, so precision is added
    to fitness.
    Input: word embedding to be evaluated
    Output: fitness score'''
    
    fitness = 0.001
    
    # restrict vocabulary for clustering
    vocab_limit, wv_limit, w2i_limit = limit_vocab(wv, w2i, vocab, exclude=gender_specific)
    
    # get clustering precision
    cluster_precision = my_cluster(wv_limit, w2i_limit, 1, vocab_limit, biased_words, num_biased_words=1000)
    fitness += (1 - cluster_precision)
    
    # build word dictionary
    #wv_dict = {}
    #for word in vocab:
        #wv_dict[word] = wv[w2i[word], :]
    
    # get categorization precision
    #cat_precision = evaluate_categorization(wv_dict, bless_data['X'], bless_data['y'])
    #fitness += cat_precision
    
    return fitness

## Algorithm Hyper-parameters

In [26]:
pop_size = 100
num_gens = 100
mutation_rate = 0.25
crossover_rate = 0.50

## Run Algorithm

In [27]:
# initialize population and evaluate fitness
population = np.random.normal(1.0, 0.1, (pop_size, 300))
fitnesses = []
for i in range(pop_size):
    # multiply word vectors by individual genome to get modified embedding
    modified_wv = glove_wv * population[i]
    fitness_i = evaluate_fitness(modified_wv, glove_w2i, glove_vocab)
    fitnesses.append(fitness_i)
print('Highest fitness after 0 generations:', max(fitnesses))

# iterate through generations
for gen in range(num_gens):
    # create array for new population
    new_population = np.zeros(population.shape)
    new_fitnesses = []
    # need to create pop_size children for next generation
    for i in range (pop_size):
        # select parent based on fitness
        index = fitness_based_selection(fitnesses)
        # random crossover
        if np.random.random() < crossover_rate:
            # select second parent
            index2 = fitness_based_selection(fitnesses)
            # combine both parents to create child
            new_population[i] = crossover(population[index], population[index2])
        else:
            # if no crossover, parent simply propogates to next gen
            new_population[i] = population[index]
        # random mutation
        if np.random.random() < mutation_rate:
            new_population[i] = mutation(new_population[i])
        # evaluate fitness
        modified_wv = glove_wv * new_population[i]
        fitness_i = evaluate_fitness(modified_wv, glove_w2i, glove_vocab)
        new_fitnesses.append(fitness_i)
    population = new_population
    fitnesses = new_fitnesses
    print('Highest fitness after {} generations: {}'.format(gen+1, max(fitnesses)))

Highest fitness after 0 generations: 0.003000000000000002
Highest fitness after 1 generations: 0.003000000000000002
Highest fitness after 2 generations: 0.003000000000000002
Highest fitness after 3 generations: 0.0034999999999999467
Highest fitness after 4 generations: 0.004000000000000003
Highest fitness after 5 generations: 0.004499999999999948
Highest fitness after 6 generations: 0.10550000000000004
Highest fitness after 7 generations: 0.10699999999999998
Highest fitness after 8 generations: 0.10750000000000004
Highest fitness after 9 generations: 0.10799999999999998
Highest fitness after 10 generations: 0.10799999999999998
Highest fitness after 11 generations: 0.10799999999999998
Highest fitness after 12 generations: 0.10799999999999998
Highest fitness after 13 generations: 0.10799999999999998
Highest fitness after 14 generations: 0.10799999999999998
Highest fitness after 15 generations: 0.10799999999999998
Highest fitness after 16 generations: 0.10799999999999998
Highest fitness a

## Evaluate fittest individual on bias and utility

In [32]:
i = np.argmax(fitnesses)
modified_wv = glove_wv * population[i]

# restrict vocabulary for clustering
vocab_limit, wv_limit, w2i_limit = limit_vocab(modified_wv, glove_w2i, glove_vocab, exclude=gender_specific)
    
# get clustering precision
cluster_precision = my_cluster(wv_limit, w2i_limit, 1, vocab_limit, biased_words, num_biased_words=1000)

In [33]:
print(cluster_precision)

0.8935


In [35]:
# build word dictionary
wv_dict = {}
for word in glove_vocab:
    wv_dict[word] = modified_wv[glove_w2i[word], :]
    
# get categorization precision
cat_precision = evaluate_categorization(wv_dict, bless_data['X'], bless_data['y'])

In [36]:
print(cat_precision)

0.8250000000000001
